In [30]:
import os
from collections import Counter, OrderedDict
import pysam
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output
from scipy.special import binom
import scipy.stats as st
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [31]:
def bam2consensus(
        ref_fname, bam_fname, ac_threshold=0, af_threshold=0):

    consensus = OrderedDict()
    for record in SeqIO.parse(ref_fname, "fasta"):
        consensus[record.id] = ["N"] * len(record)
    print(consensus.keys())

    with pysam.AlignmentFile(bam_fname, "rb") as bam:
        allele_counter = Counter()
        for pileup_column in bam.pileup():
            chrom = pileup_column.reference_name
            pos = pileup_column.reference_pos
            # print(chrom, pos)
            assert consensus[chrom][pos] == "N"

            allele_counter.clear()
            for pileup_read in pileup_column.pileups:
                if pileup_read.is_del:
                    allele = "-"
                else:
                    allele = pileup_read.alignment.query_sequence[
                        pileup_read.query_position]
                allele_counter[allele] += 1

            max_allele = "N"
            max_count, total_count = 0, 0
            for allele, count in allele_counter.items():
                if count > max_count:
                    max_count = count
                    max_allele = allele
                total_count += count

            assert max_allele in "ACGTN-"
            if (max_count >= ac_threshold and
                max_count / total_count >= af_threshold):
                consensus[chrom][pos] = max_allele

    return consensus


In [4]:
ref_fname = "refchrm.fa"
bam_fname = "bam/out.bam"

In [5]:
pysam.index(bam_fname);

In [7]:
consensus = bam2consensus(ref_fname, bam_fname)['chrM']

odict_keys(['chrM'])


In [9]:
%time
genomes = list()
genomes.append(''.join(consensus))

s = 0
for path in os.listdir('fasta'):    
    if path[-1] == 'i': #fai#
        continue
    
    path = 'fasta/'+path
    fasta_file = pysam.FastaFile(path)
    sequence = fasta_file.fetch(fasta_file.references[0])
    if s > 311: # потом убрать
        break
    if len(sequence) == 16569:
        genomes.append(sequence.upper())
        s += 1
    fasta_file.close()
genomes = np.array([list(x) for x in genomes], dtype = 'S1')

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 1.91 µs


In [10]:
%%time
same_positions = []
same_bases = []
for i in range(16569):
    if len(np.unique(genomes[:,i])) == 1:
        same_positions.append(i)
        same_bases.append(genomes[0,i])

CPU times: user 112 ms, sys: 1.95 ms, total: 114 ms
Wall time: 116 ms


In [11]:
same = dict(zip( same_positions, same_bases ))

In [12]:
%%time
correct = 0
total = 0
samfile = pysam.AlignmentFile(bam_fname, "rb" )
for pileupcolumn in tqdm(samfile.pileup("chrM"), total = 16569):
    pos = pileupcolumn.pos
    if pos not in same_positions:
        continue
    
    for pileupread in pileupcolumn.pileups:
        if not pileupread.is_del and not pileupread.is_refskip:
            total += 1
            # query position is None if is_del or is_refskip is set.
            base =  pileupread.alignment.query_sequence[pileupread.query_position]
            if base == same[pos].decode('ascii'):
                correct += 1

100%|███████████████████████████████████| 16569/16569 [00:01<00:00, 8439.00it/s]

CPU times: user 1.96 s, sys: 50.4 ms, total: 2.01 s
Wall time: 1.98 s


In [13]:
base_err = 1 - correct/total

In [32]:
%%time
num_genomes = len(genomes)
num_reads = 0
for read in samfile.fetch('chrM'):
    if not read.is_mapped:
        continue
    num_reads += 1
print(f'It\'s {num_reads} reads in bam file')
print(f'It\'s {num_genomes} genomes of possible contaminants')

16476 313
It's 16476 reads in bam file
It's 313 genomes of possible contaminants
CPU times: user 21 ms, sys: 5.49 ms, total: 26.5 ms
Wall time: 30.6 ms


In [16]:
N = np.zeros((num_reads, num_genomes))
M = np.zeros((num_reads, num_genomes))

In [17]:
%%cython -a --compile-args=-O3
cimport cython
import numpy as np
# cimport numpy as np
from tqdm import tqdm
import pysam

@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
def get_MN(int num_reads, int num_genomes,char[:, :] genomes, bam):
    cdef double[:, :] N, M
    cdef int correct, incorrect, k, i, j, pos
    cdef str seq
    N = np.zeros((num_reads, num_genomes))
    M = np.zeros((num_reads, num_genomes))
    i = 0
    j = 0
    ### ЗАПАРАЛЛЕЛИТЬ
    for read in tqdm(bam.fetch('chrM'), total = bam.count()):
        seq = read.query_sequence
        pos = read.reference_start
        if not read.is_mapped:
            continue
        for j in range(num_genomes):
            correct = 0
            incorrect = 0
            for k in range(len(seq)):
                if k+pos > 16568:
                    break
                if seq[k] == chr(genomes[j][k+pos]):
                    correct += 1
                else:
                    incorrect += 1
            M[i, j] = correct
            N[i, j] = incorrect
        i += 1
    return np.array(M, dtype=np.float64), np.array(N, dtype=np.float64)
    # pass

In [18]:
def get_MN_1(num_reads, num_genomes,genomes, bam):
    N = np.zeros((num_reads, num_genomes), dtype = np.int64)
    M = np.zeros((num_reads, num_genomes), np.int64)
    i = 0
    j = 0
    ### ЗАПАРАЛЛЕЛИТЬ
    for read in tqdm(bam.fetch('chrM'), total = bam.count()):
        seq = read.query_sequence
        pos = read.reference_start
        if not read.is_mapped:
            continue
        for j in range(num_genomes):
            correct = 0
            incorrect = 0
            for k in range(len(seq)):
                if k+pos > 16568:
                    break
                if seq[k] == genomes[j][k+pos]:
                    correct += 1
                else:
                    incorrect += 1
            M[i, j] = correct
            N[i, j] = incorrect
        i += 1
    return M, N

In [19]:
M, N = get_MN(num_reads, num_genomes, genomes, samfile)


100%|███████████████████████████████████| 16476/16476 [00:08<00:00, 2053.20it/s]


In [20]:
M = np.array(M, dtype = np.int64)
N = np.array(N, dtype = np.int64)

In [21]:
M.sum(axis=0).argmax()


0

In [24]:
# def get_Z(m, n, p, eps):
#     num_reads, num_genomes = M.shape
#     Z = np.zeros(num_reads)
#     for i in tqdm(range(num_reads), desc = f'gettitng Z'):
#         probs = np.array([Pr_De(m, n, i, j, eps) * p[j] for j in range(num_genomes)])
#         probs = probs/probs.sum()
#         Z[i] = np.random.choice(np.arange(0, num_genomes), p = probs)
#         # print(f'p = {p}\nprobs={probs}\nZ={Z[j]}')
#     return Z

In [25]:
# def get_eta(z, num_genomes):
#     return np.array([len(z[z==j]) for j in range(num_genomes)])

In [26]:
%%cython -a --compile-args=-O3
from cython.parallel import prange
cimport cython
from libc.math cimport pow
import numpy as np
from tqdm import tqdm
from scipy.special import binom


@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
def get_Z(long[:, ::1] m, long[:, ::1] n, double[::1] p,double eps):
    cdef long num_reads = m.shape[0]
    cdef long num_genomes = m.shape[1] 
    cdef long i, j
    cdef long[:] Z
    cdef double[:] probs
    cdef double s
    Z = np.empty(num_reads, dtype=int)
    for i in tqdm(range(num_reads)):
        s = 0
        probs = np.zeros(num_genomes, dtype = float)
        # with nogil:
        for j in range(num_genomes):
            probs[j] = binom(m[i, j] + n[i, j], m[i, j]) * pow((1 - eps),(m[i, j])) * pow(eps,(n[i, j])) * p[j]
            s += probs[j]
        
        for j in range(num_genomes):
            probs[j] =probs[j] / s
        
        Z[i] = np.random.choice(np.arange(0, num_genomes), p = probs)
    return Z


@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
def get_eta(long[:] z,int num_genomes):
    cdef long[:] eta
    eta = np.zeros(num_genomes, dtype = int)
    cdef int num_reads
    num_reads = z.shape[0]
    cdef int i
    for i in range(num_reads):
        eta[z[i]] += 1
    return np.array(eta)


In [27]:
# p = np.array(np.random.dirichlet([1]*num_genomes))
p = np.array([0.5] + [1/(num_genomes-1)]*(num_genomes-1))
for i in range(200):
    Z = get_Z(M, N, p, base_err)
    eta = get_eta(Z, num_genomes)
    p0 = np.random.beta(1 + eta[0],1+num_reads-eta[0])
    p_other = np.random.dirichlet(1+ eta[1:])
    p_other *= (1-p0)/p_other.sum()
    p[0] = p0
    p[1:] = p_other
    print(p)          

100%|███████████████████████████████████| 16476/16476 [00:05<00:00, 3165.28it/s]


[0.4144718  0.00107783 0.00261455 0.00231338 0.00124517 0.00196312
 0.00205298 0.00190339 0.00157242 0.00238961 0.00234362 0.00191561
 0.00191419 0.00243496 0.00236147 0.002189   0.00177424 0.00193082
 0.00163976 0.00237829 0.00232496 0.00143222 0.00248688 0.00207467
 0.00149929 0.00154975 0.00145748 0.00204729 0.00192288 0.002808
 0.00184896 0.00231936 0.00201963 0.00351361 0.00145958 0.00194075
 0.00122595 0.00138227 0.00233437 0.00134065 0.00162033 0.00132416
 0.00250138 0.00176728 0.00139485 0.00197321 0.00234612 0.00191064
 0.00226624 0.00161833 0.00289604 0.00175419 0.00187932 0.00184146
 0.00210723 0.00202339 0.00199454 0.00222437 0.00367778 0.00179335
 0.00249909 0.00307462 0.00095141 0.00158977 0.00219715 0.00190846
 0.0013147  0.00152462 0.00152547 0.00219996 0.00180048 0.00223597
 0.0008483  0.00121089 0.00131508 0.00166209 0.00191216 0.00122488
 0.00153355 0.00200679 0.00173595 0.00194839 0.00120957 0.00127076
 0.00128061 0.0022606  0.00203372 0.00206155 0.00157722 0.002774

100%|███████████████████████████████████| 16476/16476 [00:05<00:00, 3161.75it/s]


[4.91158934e-01 4.31647325e-04 2.56818266e-03 2.17529097e-03
 1.27970321e-03 2.63500089e-03 2.19351093e-03 1.22467758e-03
 2.16658948e-03 2.57816589e-03 1.61346831e-03 1.39858217e-03
 1.79645217e-03 3.12572868e-03 1.57010791e-03 9.98851635e-04
 1.84207711e-03 1.45243407e-03 1.50488026e-03 1.82115069e-03
 9.68571130e-04 1.28384933e-03 2.45864691e-03 1.58339576e-03
 1.43461246e-03 1.95289226e-03 7.70251023e-04 1.27985152e-03
 1.36090340e-03 1.76690133e-03 2.45028369e-03 1.67640837e-03
 1.35351968e-03 3.64475129e-03 1.27351603e-03 2.55557614e-03
 1.46538147e-03 6.39601664e-04 1.48291692e-03 1.11368887e-03
 1.22159896e-03 8.66870331e-04 1.62752608e-03 1.68112827e-03
 1.80826542e-03 2.51224283e-03 2.67975020e-03 1.77645041e-03
 1.55410943e-03 2.03397319e-03 2.64295488e-03 1.88345748e-03
 1.80716910e-03 1.11468731e-03 2.23640812e-03 2.03226858e-03
 1.58706636e-03 2.82844765e-03 3.78573648e-03 1.48311762e-03
 1.95675457e-03 3.41294172e-03 6.98833643e-04 1.06094809e-03
 1.45228636e-03 1.531094

100%|███████████████████████████████████| 16476/16476 [00:05<00:00, 3186.70it/s]


...

[9.12994565e-01 5.35796245e-05 3.02330466e-05 2.31874280e-04
 8.08941292e-05 4.72586302e-05 3.61986407e-05 1.06658697e-04
 2.54830996e-04 6.15087760e-05 1.61720339e-05 3.77771402e-04
 4.11949987e-04 1.16022433e-05 1.23420174e-04 4.43969486e-04
 4.64204989e-05 7.88995834e-04 2.61056482e-04 1.21546933e-04
 5.53374027e-05 1.24496382e-04 2.80738058e-04 2.00271602e-05
 3.58548766e-04 4.43654875e-04 3.71751362e-04 6.52544803e-05
 1.81052096e-04 1.75280098e-04 5.47122344e-05 6.36504812e-06
 5.96784809e-05 1.94086197e-04 1.13573021e-04 3.53643554e-05
 2.84583675e-04 3.65870202e-04 7.49459019e-05 8.37255987e-04
 2.64308245e-04 4.49135290e-05 2.64904984e-04 7.55930385e-06
 7.10026348e-04 1.22035024e-04 1.47019807e-06 9.57090847e-04
 3.79354750e-04 7.02883053e-05 2.95478341e-04 1.05563607e-04
 9.07339719e-05 2.55867073e-04 3.45689494e-04 4.88904011e-05
 1.04761862e-05 3.68196127e-05 1.48439810e-04 7.99604459e-05
 3.19685439e-05 2.90914622e-04 3.81847585e-05 3.51780565e-04
 4.39573826e-05 2.736549

100%|███████████████████████████████████| 16476/16476 [00:05<00:00, 3169.04it/s]


[9.12796011e-01 1.39515862e-04 1.99668254e-04 1.23206907e-04
 3.45967636e-05 2.29573475e-05 5.80414289e-05 7.73495603e-05
 4.12378454e-04 6.58053274e-05 1.97760716e-04 3.69175870e-04
 1.23631737e-04 1.66688974e-05 2.63723879e-04 3.43195668e-04
 3.22560758e-05 8.43378104e-04 2.21044997e-05 4.48766235e-05
 6.58190298e-05 9.01811768e-05 2.16073599e-04 2.47338150e-06
 4.96494724e-04 3.61226797e-04 1.02387007e-03 3.25745087e-05
 1.54689129e-04 2.43299161e-05 2.80994583e-05 2.16201210e-05
 1.02713571e-04 1.10156713e-04 5.66918473e-05 1.57130673e-05
 2.55931731e-04 3.82398607e-04 1.05955304e-04 6.67512450e-04
 1.49136920e-04 2.55670316e-05 6.32154133e-05 1.08089127e-04
 4.42030451e-04 1.32448189e-04 1.03762326e-04 8.98871804e-04
 2.74772535e-04 5.24997578e-05 5.46363396e-05 1.28485333e-04
 1.54034493e-04 1.40842550e-04 8.04265826e-05 2.90673550e-04
 2.14114792e-05 5.83908167e-06 1.06092016e-05 3.06236854e-05
 1.47005804e-04 3.00429490e-04 8.55624210e-05 3.24856628e-04
 5.04353894e-05 2.382364

 60%|█████████████████████▋              | 9913/16476 [00:03<00:02, 3168.79it/s]


KeyboardInterrupt: 

In [ ]:
for record in SeqIO.parse('311humans.fasta', "fasta"):
    print(len((str(record.seq)).replace('-','')))